## Pytorch Cifar-10 Classifier

This is a mini-project implementing Cifar-10 classifier using PyTorch and the concept of Wide Residual Network. 

### Importing necessary libraries and tools:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

import os
import argparse
import random
from model_wrn import wrn

### Preprocessing the data using augmentation and normalization (For testing data there is only normalization implemented):

In [2]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

==> Preparing data..


### Detect if there is available GPU:

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Load training images of CIFAR-10 twice: one for training, one for validation, with different preprocessing function:


In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

1.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

6.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

11.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

20.1%IOPub m

### Learning Rate Schedule (Step Dacay):

In [6]:
def adjust_learning_rate(optimizer, epoch):
    if(epoch<61):
        lr=0.1
    elif(epoch<121):
        lr=0.02
    elif(epoch<161):
        lr=0.004
    elif(epoch<=200):
        lr=0.0008
    #print(lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### Defining training function:

In [7]:
def train(epoch,dataLoader,model):
    adjust_learning_rate(optimizer, epoch)

    print('Epoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataLoader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

### Defining testing function, the best model will be stored in ./checkpoint-no-cross/ and continuously updated: 

In [9]:
def test(epoch,dataLoader,model):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    batch_num=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataLoader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_num+=1

    acc = 100.*correct/total
    print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_num), 100.*correct/total, correct, total))
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint-no-cross'):
            os.mkdir('checkpoint-no-cross')
        torch.save(state, './checkpoint-no-cross/' + 'ckpt{}.pth'.format(0))
        best_acc = acc

### Using torchsummary to examine the number of parameters in the model:

In [11]:
import torchsummary

net = wrn(
          num_classes=10,
          depth=16,
          widen_factor=5,
          dropRate=0.3,
        )
net.cuda()
torchsummary.summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 80, 32, 32]          11,520
       BatchNorm2d-5           [-1, 80, 32, 32]             160
              ReLU-6           [-1, 80, 32, 32]               0
            Conv2d-7           [-1, 80, 32, 32]          57,600
            Conv2d-8           [-1, 80, 32, 32]           1,280
        BasicBlock-9           [-1, 80, 32, 32]               0
      BatchNorm2d-10           [-1, 80, 32, 32]             160
             ReLU-11           [-1, 80, 32, 32]               0
           Conv2d-12           [-1, 80, 32, 32]          57,600
      BatchNorm2d-13           [-1, 80, 32, 32]             160
             ReLU-14           [-1, 80,

### Training process:

In [12]:

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=2)

print('================> Building model {}..'.format(0))
net =wrn(
      num_classes=10,
      depth=16,
      widen_factor=5,
      dropRate=0.3,
    )
net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True


#Initialize Optimizer
best_acc = 0  # best test accuracy
start_epoch = 0
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

#training and testing
for epoch in range(start_epoch, start_epoch + 200):
    train(epoch,train_loader,net)
    test(epoch,val_loader,net)

================> Building model 0..
Epoch: 0
Loss: 1.595 | Acc: 50.460% (5046/10000)
Saving..
Epoch: 1
Loss: 0.906 | Acc: 67.790% (6779/10000)
Saving..
Epoch: 2
Loss: 0.898 | Acc: 70.420% (7042/10000)
Saving..
Epoch: 3
Loss: 0.806 | Acc: 72.940% (7294/10000)
Saving..
Epoch: 4
Loss: 0.719 | Acc: 75.790% (7579/10000)
Saving..
Epoch: 5
Loss: 0.682 | Acc: 76.860% (7686/10000)
Saving..
Epoch: 6
Loss: 0.866 | Acc: 72.170% (7217/10000)
Epoch: 7
Loss: 0.583 | Acc: 80.320% (8032/10000)
Saving..
Epoch: 8
Loss: 0.602 | Acc: 79.250% (7925/10000)
Epoch: 9
Loss: 0.625 | Acc: 79.050% (7905/10000)
Epoch: 10
Loss: 0.552 | Acc: 81.800% (8180/10000)
Saving..
Epoch: 11
Loss: 0.653 | Acc: 79.030% (7903/10000)
Epoch: 12
Loss: 0.708 | Acc: 77.730% (7773/10000)
Epoch: 13
Loss: 0.714 | Acc: 77.220% (7722/10000)
Epoch: 14
Loss: 0.666 | Acc: 79.840% (7984/10000)
Epoch: 15
Loss: 0.656 | Acc: 79.800% (7980/10000)
Epoch: 16
Loss: 0.563 | Acc: 81.590% (8159/10000)
Epoch: 17
Loss: 0.455 | Acc: 84.680% (8468/10000)
S

Loss: 0.201 | Acc: 94.590% (9459/10000)
Epoch: 160
Loss: 0.213 | Acc: 94.360% (9436/10000)
Epoch: 161
Loss: 0.194 | Acc: 94.710% (9471/10000)
Saving..
Epoch: 162
Loss: 0.194 | Acc: 94.640% (9464/10000)
Epoch: 163
Loss: 0.194 | Acc: 94.740% (9474/10000)
Saving..
Epoch: 164
Loss: 0.193 | Acc: 94.670% (9467/10000)
Epoch: 165
Loss: 0.193 | Acc: 94.730% (9473/10000)
Epoch: 166
Loss: 0.196 | Acc: 94.620% (9462/10000)
Epoch: 167
Loss: 0.194 | Acc: 94.780% (9478/10000)
Saving..
Epoch: 168
Loss: 0.192 | Acc: 94.810% (9481/10000)
Saving..
Epoch: 169
Loss: 0.194 | Acc: 94.710% (9471/10000)
Epoch: 170
Loss: 0.192 | Acc: 94.780% (9478/10000)
Epoch: 171
Loss: 0.193 | Acc: 94.770% (9477/10000)
Epoch: 172
Loss: 0.193 | Acc: 94.730% (9473/10000)
Epoch: 173
Loss: 0.193 | Acc: 94.690% (9469/10000)
Epoch: 174
Loss: 0.193 | Acc: 94.670% (9467/10000)
Epoch: 175
Loss: 0.193 | Acc: 94.720% (9472/10000)
Epoch: 176
Loss: 0.192 | Acc: 94.590% (9459/10000)
Epoch: 177
Loss: 0.194 | Acc: 94.740% (9474/10000)
Epoch:

### Testing of the final result, the best model is stored in ./checkpoint-no-cross: 

In [12]:
model_dir = "./checkpoint-no-cross/"

net =wrn(
      num_classes=10,
      depth=16,
      widen_factor=5,
      dropRate=0.3,
    )
net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    checkpoint = torch.load(model_dir+'ckpt{}.pth'.format(0))
    net.load_state_dict(checkpoint['net'])

net.eval()
    
correct = 0
total = 0
    
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    print("Accuracy: %.3f"%(acc))

Accuracy: 95.060
